In [0]:
import re
from collections import defaultdict
from os import listdir
from os.path import isfile
import random

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import numpy as np

In [0]:
MAX_DOC_LENGTH = 500  # do dai lon nhat cua 1 document
NUM_CLASSES = 20

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# Thu thap du lieu tu files va xay dung tu dien
def gen_data_and_vocab():
    def collect_data_from(parent_path, newsgroup_list, word_count=None):
        data = []  # chua noi dung cac van ban
        for group_id, newsgroup in enumerate(newsgroup_list):
            dir_path = parent_path + '/' + newsgroup + '/'
            files = [(filename, dir_path + filename)
                     for filename in listdir(dir_path)
                     if isfile(dir_path + filename)]
            files.sort()  # sap xep theo filename
            label = group_id
            print('Processing: {}-{}'.format(group_id, newsgroup))
            for filename, filepath in files:
                with open(filepath, 'rb') as f:
                    text= f.read().decode('UTF-8', errors='ignore').lower()
                    words = re.split('\W+', text)
                    if word_count is not None: # chi ap dung cho train data 
                        for word in words:
                            word_count[word] += 1
                    content = ' '.join(words)
                    assert len(content.splitlines()) == 1
                    data.append(str(label) + '<fff>' + filename + '<fff>' + content)
        return data
  
    word_count = defaultdict(int)
    path = '/content/drive/My Drive/DSLabTraining/Datasets/20news-bydate/'
    parts = [path + dir_name + '/' for dir_name in listdir(path)
             if not isfile(path + dir_name)]
    train_path, test_path = (parts[0], parts[1]) \
        if 'train' in parts[0] else (parts[1], parts[0])
    newsgroup_list = [newsgroup for newsgroup in listdir(train_path)]
    newsgroup_list.sort()

    train_data = collect_data_from(
        parent_path=train_path,
        newsgroup_list=newsgroup_list,
        word_count=word_count
    )
    # tu dien bao gom nhung tu co tan suat xuat hien du lon
    vocab = [word for word, fred in
             zip(word_count.keys(), word_count.values()) if fred > 10]
    vocab.sort()
    with open('/content/drive/My Drive/DSLabTraining/Datasets/w2v/vocab-raw.txt', 'w') as f:
        f.write('\n'.join(vocab))

    test_data = collect_data_from(
        parent_path=test_path,
        newsgroup_list=newsgroup_list
    )

    with open('/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-train-raw.txt', 'w') as f:
        f.write('\n'.join(train_data))
    with open('/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-test-raw.txt', 'w') as f:
        f.write('\n'.join(test_data))



In [0]:
# encode du lieu
# Ma hoa moi van ban boi ID cua cac tu co trong van ban do
def encode_data(data_path, vocab_path):  # data_path chi den file 20news-train-raw.txt va file 20news-test-raw
    with open(vocab_path) as f:
        vocab = dict([(word, word_ID + 2)
                      for word_ID, word in enumerate(f.read().splitlines())])
    unknown_ID = 0  # ID cua tu khong xuat hien trong vocab
    padding_ID = 1  # ID cua tu rong trong cac van ban
    with open(data_path) as f:
        documents = [(line.split('<fff>')[0], line.split('<fff>')[1], line.split('<fff>')[2])
                     for line in f.read().splitlines()]
    encoded_data = []
    # lay thong tin tu cac van ban
    for document in documents:
        label, doc_id, text = document
        words = text.split()[:MAX_DOC_LENGTH]
        sentence_length = len(words)
        encoded_text = []

        for word in words:
            if word in vocab:
                encoded_text.append(str(vocab[word]))
            else:
                encoded_text.append(str(unknown_ID))
        if sentence_length < MAX_DOC_LENGTH:
            num_padding = MAX_DOC_LENGTH - sentence_length
            for _ in range(num_padding):
                encoded_text.append(str(padding_ID))
        encoded_data.append(str(label) + '<fff>' + str(doc_id) + '<fff>' + str(sentence_length)
                            + '<fff>' + ' '.join(encoded_text))
    dir_name = '/'.join(data_path.split('/')[:-1])
    file_name = '-'.join(data_path.split('/')[-1].split('-')[:-1]) + '-encoded.txt'
    with open(dir_name + '/' + file_name, 'w') as f:
        f.write('\n'.join(encoded_data))

In [0]:
class DataReader:
    def __init__(self, data_path, batch_size):
        self._batch_size = batch_size
        with open(data_path) as f:
            d_lines = f.read().splitlines()
        self._data = []
        self._labels = []
        self._sentence_lengths = []
        for _, line in enumerate(d_lines):
          if len(line) > 1:
            features = line.split("<fff>")
            label, sentence_length = int(features[0]), int(features[2])
            tokens = features[3].split()
            vector = [int(token) for token in tokens]
            self._data.append(vector)
            self._labels.append(label)
            self._sentence_lengths.append(sentence_length)
        self._data = np.array(self._data)
        self._labels = np.array(self._labels)
        self._sentence_lengths = np.array(self._sentence_lengths)
        
        self._num_epoch = 0
        self._batch_id = 0

    def next_batch(self):
        start = self._batch_id * self._batch_size
        end = start + self._batch_size
        self._batch_id += 1

        if end + self._batch_size > len(self._data):
            end = len(self._data)
            start = end - self._batch_size
            self._num_epoch += 1
            self._batch_id = 0
            indices = list(range(len(self._data)))
            random.seed(2020)
            random.shuffle(indices)
            self._data, self._labels, self._sentence_lengths= self._data[indices], self._labels[indices], self._sentence_lengths[indices]

        return self._data[start:end], self._labels[start:end], self._sentence_lengths[start:end]

In [0]:
class RNN:
    def __init__(self, vocab_size, embedding_size, lstm_size, batch_size):
        self._vocab_size = vocab_size
        self._embedding_size = embedding_size
        self._lstm_size = lstm_size
        self._batch_size = batch_size

        self._data = tf.placeholder(tf.int32, shape=[batch_size, MAX_DOC_LENGTH])
        self._labels = tf.placeholder(tf.int32, shape=[batch_size, ])
        self._sentence_lengths = tf.placeholder(tf.int32, shape=[batch_size, ])
        # self._final_tokens = tf.placeholder(tf.int32, shape=[batch_size, ])

    def build_graph(self):
        embeddings = self.embedding_layer(self._data)
        lstm_outputs = self.LSTM_layer(embeddings)
        
        with tf.variable_scope("rnn_variables", reuse= tf.AUTO_REUSE) as scope:
          weights = tf.get_variable(
              name='final_layer_weights',
              shape=(self._lstm_size, NUM_CLASSES),
              initializer=tf.random_normal_initializer(seed=2020)
          )
          biases = tf.get_variable(
              name='final_layer_biases',
              shape=(NUM_CLASSES),
              initializer=tf.random_normal_initializer(seed=2020)
          )

          logits = tf.matmul(lstm_outputs, weights) + biases
          labels_one_hot = tf.one_hot(
              indices=self._labels,
              depth=NUM_CLASSES,
              dtype=tf.float32
          )
          loss = tf.nn.softmax_cross_entropy_with_logits(
              labels=labels_one_hot,
              logits=logits
          )
          loss = tf.reduce_mean(loss)

          probs = tf.nn.softmax(logits)
          predicted_labels = tf.argmax(probs, axis=1)
          predicted_labels = tf.squeeze(predicted_labels)

        return predicted_labels, loss

    def embedding_layer(self, indices):
        pretrained_vectors = []
        pretrained_vectors.append(np.zeros(self._embedding_size))
        np.random.seed(2020)
        for _ in range(self._vocab_size + 1):
            pretrained_vectors.append(np.random.normal(loc=0., scale=1., size=self._embedding_size))

        pretrained_vectors = np.array(pretrained_vectors)
        with tf.variable_scope("rnn_variables", reuse=tf.AUTO_REUSE) as scope:
          self._embedding_matrix = tf.get_variable(
              name='embedding',
              shape=(self._vocab_size + 2, self._embedding_size),
              initializer=tf.constant_initializer(pretrained_vectors)
          )
        return tf.nn.embedding_lookup(self._embedding_matrix, indices)

    def LSTM_layer(self, embeddings):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(self._lstm_size)
        zero_state = tf.zeros(shape=(self._batch_size, self._lstm_size))
        initial_state = tf.nn.rnn_cell.LSTMStateTuple(zero_state, zero_state)

        lstm_inputs = tf.unstack(tf.transpose(embeddings, perm=[1, 0, 2]))
        lstm_outputs, last_state = tf.nn.static_rnn(
            cell=lstm_cell,
            inputs=lstm_inputs,
            initial_state=initial_state,
            sequence_length=self._sentence_lengths
        )

        lstm_outputs = tf.unstack(tf.transpose(lstm_outputs, perm=[1, 0, 2]))
        lstm_outputs = tf.concat(
            lstm_outputs,
            axis=0
        )  # [num docs * MAX_SENT_LENGTH,  ]

        # self._mask : [num docs * MAX_SENT_LENGTH, ]
        mask = tf.sequence_mask(
            lengths=self._sentence_lengths,
            maxlen=MAX_DOC_LENGTH,
            dtype=tf.float32
        )  # [num_docs, MAX_SENTENCE_LENGTH]
        mask = tf.concat(tf.unstack(mask, axis=0), axis=0)
        mask = tf.expand_dims(mask, -1)
        lstm_outputs = mask * lstm_outputs
        lstm_outputs_split = tf.split(lstm_outputs, num_or_size_splits=self._batch_size)
        lstm_outputs_sum = tf.reduce_sum(lstm_outputs_split, axis=1)  # [num_docs, lsmt_size]
        lstm_outputs_average = lstm_outputs_sum / tf.expand_dims(tf.cast(self._sentence_lengths, tf.float32), -1)
        return lstm_outputs_average

    def trainer(self, loss, learning_rate):
        with tf.variable_scope("rnn_variables", reuse= tf.AUTO_REUSE) as scope:
          train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        return train_op


In [0]:
def train_and_evaluate_RNN():
    with open('/content/drive/My Drive/DSLabTraining/Datasets/w2v/vocab-raw.txt') as f:
        vocab_size = len(f.read().splitlines())
    tf.set_random_seed(2020)
    rnn = RNN(
        vocab_size=vocab_size,
        embedding_size=300,
        lstm_size=50,
        batch_size=50
    )
    predicted_labels, loss = rnn.build_graph()
    train_op = rnn.trainer(loss=loss, learning_rate=0.01)

    with tf.Session() as sess:
        train_data_reader = DataReader(
            data_path='/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-train-encoded.txt',
            batch_size=50
        )
        test_data_reader = DataReader(
            data_path='/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-test-encoded.txt',
            batch_size=50
        )
        step = 0
        MAX_STEP = 2000
        sess.run(tf.global_variables_initializer())

        while step < MAX_STEP:
            next_train_batch = train_data_reader.next_batch()
            train_data, train_labels, train_sentence_lengths = next_train_batch
            plabels_eval, loss_eval, _ = sess.run(
                [predicted_labels, loss, train_op],
                feed_dict={
                    rnn._data: train_data,
                    rnn._labels: train_labels,
                    rnn._sentence_lengths: train_sentence_lengths,
                    # rnn._final_tokens: train_final_tokens
                }
            )
            step += 1
            if step % 50 == 0:
                print('loss after step {} is {}'.format(step, loss_eval))

            if train_data_reader._batch_id == 0:
                num_true_preds = 0
                while True:
                    next_test_batch = test_data_reader.next_batch()
                    test_data, test_labels, test_sentence_lengths = next_test_batch

                    test_plabels_eval = sess.run(
                        predicted_labels,
                        feed_dict={
                            rnn._data: test_data,
                            rnn._labels: test_labels,
                            rnn._sentence_lengths: test_sentence_lengths,
                            # rnn._final_tokens: test_final_tokens
                        }
                    )
                    matches = np.equal(test_plabels_eval, test_labels)
                    num_true_preds += np.sum(matches.astype(float))

                    if test_data_reader._batch_id == 0:
                        break

                print('Epoch:', train_data_reader._num_epoch)
                print('Accuracy on test data:', num_true_preds * 100. / len(test_data_reader._data))



In [9]:
if __name__=="__main__":
    #gen_data_and_vocab()

    #train_data_path='/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-train-raw.txt'
    #test_data_path='/content/drive/My Drive/DSLabTraining/Datasets/w2v/20news-test-raw.txt'
    #vocab_path='/content/drive/My Drive/DSLabTraining/Datasets/w2v/vocab-raw.txt'
    #encode_data(train_data_path, vocab_path)
    #encode_data(test_data_path, vocab_path)
    
    with tf.variable_scope("rnn_variables", reuse= tf.AUTO_REUSE) as scope:
        train_and_evaluate_RNN()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

loss after step 50 is 2.987929344177246
loss after step 100 is 1.4307098388671875
loss after step 150 is 0.5380045175552368
loss after step 200 is 5.457465171813965
Epoch: 1
Accuracy on test data: 7.221558476038763
loss after step 250 is 2.7571749687194824
loss after step 300 is 2.1142306327819824
